In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib widget

In [ ]:
from lsst.utils.plotting import publication_plots
from lsst.utils.plotting import get_multiband_plot_colors

colors = get_multiband_plot_colors()
bands = colors.keys()

In [ ]:
from lsst.summit.utils import (
    getAirmassSeeingCorrection,
    getBandpassSeeingCorrection,
)

In [ ]:
CAM_FWHM = 0.207 # arcsec

In [ ]:
#import os
#os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
instrument = 'lsstcam'

In [ ]:
def fetch(day_obs_min, day_obs_max):
    
    visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.psf_sigma,
    ccdvisit1_quicklook.psf_ixx,
    ccdvisit1_quicklook.psf_ixy,
    ccdvisit1_quicklook.psf_iyy,
    --ccdvisit1_quicklook.z4,
    --ccdvisit1_quicklook.z5,
    --ccdvisit1_quicklook.z6,
    --ccdvisit1_quicklook.z7,
    --ccdvisit1_quicklook.z8,
    --ccdvisit1_quicklook.z9,
    --ccdvisit1_quicklook.z10,
    --ccdvisit1_quicklook.z11,
    --ccdvisit1_quicklook.z12,
    --ccdvisit1_quicklook.z13,
    --ccdvisit1_quicklook.z14,
    --ccdvisit1_quicklook.z15,
    --ccdvisit1_quicklook.z16,
    --ccdvisit1_quicklook.z17,
    --ccdvisit1_quicklook.z18,
    --ccdvisit1_quicklook.z19,
    --ccdvisit1_quicklook.z20,
    --ccdvisit1_quicklook.z21,
    --ccdvisit1_quicklook.z22,
    --ccdvisit1_quicklook.z23,
    --ccdvisit1_quicklook.z24,
    --ccdvisit1_quicklook.z25,
    --ccdvisit1_quicklook.z26,
    --ccdvisit1_quicklook.z27,
    --ccdvisit1_quicklook.z28,
    ccdvisit1.detector,
    visit1.visit_id,
    visit1.seq_num,
    visit1.band,
    visit1.physical_filter,
    visit1.day_obs,
    visit1.target_name,
    visit1.science_program,
    visit1.observation_reason,
    visit1.airmass,
    visit1.altitude,
    visit1.azimuth,
    visit1.sky_rotation,
    visit1.exp_midpt_mjd,
    visit1.dimm_seeing,
    visit1.s_ra,
    visit1.s_dec,
    visit1_quicklook.psf_sigma_min,
    visit1_quicklook.psf_sigma_median,
    visit1_quicklook.aos_fwhm,
    visit1_quicklook.donut_blur_fwhm,
    visit1_quicklook.ringss_seeing,
    visit1_quicklook.seeing_zenith_500nm_min,
    visit1_quicklook.seeing_zenith_500nm_median,
    visit1_quicklook.physical_rotator_angle
    FROM
    cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_{instrument}.ccdvisit1 AS ccdvisit1,
    cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
    cdb_{instrument}.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.visit_id = visit1_quicklook.visit_id
    AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161) -- vignetted
    AND ccdvisit1.detector NOT IN (191, 192, 195, 196, 199, 200, 203, 204) -- corner wavefront sensors
    AND visit1.airmass > 0
    AND visit1.day_obs >= {day_obs_min} AND visit1.day_obs <= {day_obs_max}
    -- AND visit1.science_program in ('BLOCK-365', 'BLOCK-407');
    AND visit1.science_program in ('BLOCK-365', 'BLOCK-407', 'BLOCK-408', 'BLOCK-416', 'BLOCK-417', 'BLOCK-419');
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    pixel_scale = 0.2
    sig2fwhm = 2 * np.sqrt(2 * np.log(2))
    ccdvisits["psf_fwhm"] = ccdvisits["psf_sigma"] * sig2fwhm * pixel_scale
    ccdvisits["psf_fwhm"] = pd.to_numeric(ccdvisits["psf_fwhm"], errors="coerce")
    ccdvisits["donut_blur_fwhm"] = pd.to_numeric(ccdvisits["donut_blur_fwhm"], errors="coerce")
    ccdvisits["aos_fwhm"] = pd.to_numeric(ccdvisits["aos_fwhm"], errors="coerce")
    
    #airmass_seeing_correction = np.array([getAirmassSeeingCorrection(airmass) for airmass in ccdvisits["airmass"]])
    #bandpass_seeing_correction = np.array([getBandpassSeeingCorrection(band) for band in ccdvisits["physical_filter"]])
    #ccdvisits["psf_fwhm_zenith_500nm"] = ccdvisits["psf_fwhm"] * airmass_seeing_correction * bandpass_seeing_correction

    #ccdvisits["psf_e1"] = (ccdvisits["psf_ixx"]**2 - ccdvisits["psf_iyy"]**2) / (ccdvisits["psf_ixx"]**2 + ccdvisits["psf_iyy"]**2)
    #ccdvisits["psf_e2"] = (2. * ccdvisits["psf_ixy"]**2) / (ccdvisits["psf_ixx"]**2 + ccdvisits["psf_iyy"]**2)
    ccdvisits["psf_e1"] = (ccdvisits["psf_ixx"] - ccdvisits["psf_iyy"]) / (ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"])
    ccdvisits["psf_e2"] = (2. * ccdvisits["psf_ixy"]) / (ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"])
    ccdvisits["psf_e"] = np.sqrt(np.array(ccdvisits["psf_e1"]**2 + ccdvisits["psf_e2"]**2, dtype=float))

    ccdvisits["psf_e"] = pd.to_numeric(ccdvisits["psf_e"], errors="coerce")
    ccdvisits["psf_e1"] = pd.to_numeric(ccdvisits["psf_e1"], errors="coerce")
    ccdvisits["psf_e2"] = pd.to_numeric(ccdvisits["psf_e2"], errors="coerce")

    return ccdvisits

In [ ]:
#day_obs_min = 20250620
#day_obs_max = 20250709
#day_obs_min = 20250906
#day_obs_max = 20250906
#day_obs_min = 20250708
#day_obs_max = 20250708
#day_obs_min = 20250709
#day_obs_max = 20250709
#day_obs_min = 20250913
#day_obs_max = 20250913
#day_obs_min = 20250915
#day_obs_max = 20250915
#day_obs_min = 20250921
#day_obs_max = 20250921
day_obs_min = 20251026
#day_obs_max = 20251026
#day_obs_min = 20251027
#day_obs_max = 20251027
#day_obs_min = 20251029
#day_obs_max = 20251029
#day_obs_max = 20251102
#day_obs_min = 20251104
#day_obs_max = 20251125
#day_obs_max = 20251201
#day_obs_min = 20251114
#day_obs_min = 20251126
#day_obs_min = 20251128
#day_obs_max = 20251231
#day_obs_min = 20260106
#day_obs_min = 20251230
day_obs_max = 20260107
ccdvisits = fetch(day_obs_min, day_obs_max)

In [ ]:
airmass_seeing_correction = np.array([getAirmassSeeingCorrection(airmass) for airmass in ccdvisits["airmass"]])
bandpass_seeing_correction = np.array([getBandpassSeeingCorrection(band) for band in ccdvisits["physical_filter"]])
ccdvisits["psf_fwhm_zenith_500nm"] = ccdvisits["psf_fwhm"] * airmass_seeing_correction * bandpass_seeing_correction

In [ ]:
ccdvisits['donut_blur_atm_fwhm'] = np.sqrt(ccdvisits['donut_blur_fwhm']**2 - CAM_FWHM**2)
ccdvisits['aos_cam_fwhm'] = np.sqrt(ccdvisits['aos_fwhm']**2 + CAM_FWHM**2)

In [ ]:
ccdvisits['target_name'].str.contains('ddf')

In [ ]:
bins = np.linspace(0.5, 3., 50)

selection = ccdvisits['target_name'].str.contains('ddf') & (np.fabs(ccdvisits['physical_rotator_angle']) < 20.)
selection = ccdvisits['target_name'].str.contains('ddf') & (np.fabs(ccdvisits['physical_rotator_angle']) > 20.)
#selection_2 = ccdvisits['target_name'].str.contains('ddf') & (ccdvisits['airmass'] > 1.2)

#print(np.sum(selection) / 181)
#print(np.sum(selection_2) / 181)

print(np.unique(ccdvisits['target_name'][selection]))

plt.figure()
plt.hist(ccdvisits["psf_fwhm"][selection], bins=bins, histtype='step', density=True, label='DDF')
plt.hist(ccdvisits["psf_fwhm"][selection_2], bins=bins, histtype='step', density=True, label='DDF')
#plt.hist(ccdvisits["psf_fwhm"][~selection], bins=bins, histtype='step', density=True, label='non-DDF')
plt.legend()
plt.title('FBS Visits: %s - %s'%(day_obs_min, day_obs_max))
plt.xlabel('PSF FWHM (arcsec)')
plt.ylabel('PDF')



In [ ]:
selection = ccdvisits['target_name'].str.contains('ddf')

plt.figure()
for band in bands:
    band_selection = selection & (ccdvisits["band"] == band)
    plt.scatter(
        ccdvisits["seq_num"][selection & band_selection], 
        ccdvisits["psf_fwhm"][selection & band_selection],
        c=colors[band], label=band,
        edgecolor='none', s=1
    )

In [ ]:
ccdvisits

In [ ]:
np.sort(ccdvisits['seq_num'].unique())

In [ ]:
selection = (ccdvisits['band'] != "y") & (ccdvisits['day_obs'] >= 20251026)
print(np.sum(selection))

plt.figure()
plt.hexbin(
    ccdvisits["donut_blur_atm_fwhm"][selection], 
    ccdvisits["psf_fwhm"][selection], 
    #ccdvisits["psf_fwhm_zenith_500nm"][selection], 
    gridsize=50, mincnt=10, extent=(0.5, 2.0, 0.5, 2.0), cmap='viridis')
plt.colorbar(label='Density')
plt.plot(np.linspace(0.5, 2.0, 10), np.linspace(0.5, 2.0, 10), lw=2, ls='--', c='black')

x = np.linspace(0.5, 2.0, 100)
plt.plot(x, np.sqrt(x**2 + 0.4**2), c='red', lw=2, ls='--', label='0.4" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.5**2), c='orange', lw=2, ls='--', label='0.5" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.6**2), c='cyan', lw=2, ls='--', label='0.6" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.7**2), c='magenta', lw=2, ls='--', label='0.7" System Contribution')

plt.xlim(0.5, 1.8)
plt.ylim(0.5, 1.8)

plt.legend(loc='lower right')

#plt.xlabel("$\sqrt{ {\rm Donut_Blur_FWHM}^2 - {\rm CAM_FWHM}^2}$ (arcsec)")
plt.xlabel("$\sqrt{\mathrm{DONUT\_BLUR\_FWHM}^2 - \mathrm{CAM\_FWHM}^2}$ (arcsec)")
#plt.ylabel("psf_fwhm_zenith_500nm")
plt.ylabel("PSF FWHM (arcsec)")
plt.title("Using Donut Blur as Proxy for Atmosphere Contribution")
#plt.show()

In [ ]:
bins = np.linspace(0., 2., 41)
plt.figure()
plt.hist(
    np.sqrt(ccdvisits["psf_fwhm"][selection]**2 + CAM_FWHM**2 - ccdvisits["donut_blur_fwhm"][selection]**2), 
    bins=bins, histtype='step', label='w/ CAM_FWHM',
)
plt.hist(
    np.sqrt(ccdvisits["psf_fwhm"][selection]**2 - ccdvisits["donut_blur_fwhm"][selection]**2), 
    bins=bins, histtype='step', label='w/out CAM_FWHM',
)
plt.legend()

In [ ]:
bins = np.linspace(0., 2., 41)
plt.figure()
plt.hist(
    ccdvisits["aos_fwhm"][selection], 
    bins=bins, histtype='step', label='w/out CAM_FWHM',
)
plt.hist(
    ccdvisits["aos_cam_fwhm"], 
    bins=bins, histtype='step', label='w/ CAM_FWHM',
)
plt.legend()

In [ ]:
"""
selection = (ccdvisits['seq_num'] == 52) 

bins = np.linspace(-0.5, 0.5, 150)
plt.figure()
plt.hist(ccdvisits['psf_e1'][selection], bins=bins, histtype='step', label='e1')
plt.hist(ccdvisits['psf_e2'][selection], bins=bins, histtype='step', label='e2')
plt.hist(ccdvisits['psf_e'][selection], bins=bins, histtype='step', label='e')
plt.legend()
"""

In [ ]:
selection = (ccdvisits['day_obs'] >= 20251130) & (ccdvisits['day_obs'] <= 20251202)
print(np.sum(selection) / 189.)


bins = np.linspace(-0.4, 0.4, 81)

plt.figure()
plt.hist(ccdvisits['psf_e'][selection], bins=bins, density=True, histtype='step', lw=2, label='PSF e')
plt.hist(ccdvisits['psf_e1'][selection], bins=bins, density=True, histtype='step', lw=2, label='PSF e1')
plt.hist(ccdvisits['psf_e2'][selection], bins=bins, density=True, histtype='step', lw=2, label='PSF e2')
plt.legend()
plt.xlabel('Ellipticity')
plt.ylabel('PDF')
plt.xlim(bins[0], bins[-1])
plt.ylim(0., plt.ylim()[-1])
plt.title('day_obs: %i - %s'%(min(ccdvisits['day_obs'][selection]), max(ccdvisits['day_obs'][selection])))
plt.show()

In [ ]:
groups = ccdvisits.groupby('visit_id')
visits_summary = pd.DataFrame({
    'day_obs': groups['day_obs'].first(),
    'target_name': groups['target_name'].first(),
    'science_program': groups['science_program'].first(),
    'observation_reason': groups['observation_reason'].first(),
    'seq_num': groups['seq_num'].median(),
    'exp_midpt_mjd': groups['exp_midpt_mjd'].median(),
    'donut_blur_fwhm': groups['donut_blur_fwhm'].median(),
    'aos_fwhm': groups['aos_fwhm'].median(),
    'donut_blur_atm_fwhm': groups['donut_blur_atm_fwhm'].median(),
    'aos_cam_fwhm': groups['aos_cam_fwhm'].median(),
    'physical_rotator_angle': groups['physical_rotator_angle'].median(),
    'altitude': groups['altitude'].median(),
    'psf_fwhm_05': groups['psf_fwhm'].quantile(0.05),
    'psf_fwhm_50': groups['psf_fwhm'].quantile(0.50),
    'psf_fwhm_95': groups['psf_fwhm'].quantile(0.95),
    'psf_fwhm_zenith_500nm_50': groups['psf_fwhm_zenith_500nm'].quantile(0.50),
    'psf_e_05': groups['psf_e'].quantile(0.05),
    'psf_e_50': groups['psf_e'].quantile(0.50),
    'psf_e_95': groups['psf_e'].quantile(0.95),
    'psf_e1_05': groups['psf_e1'].quantile(0.05),
    'psf_e1_50': groups['psf_e1'].quantile(0.50),
    'psf_e1_95': groups['psf_e1'].quantile(0.95),
    'psf_e2_05': groups['psf_e2'].quantile(0.05),
    'psf_e2_50': groups['psf_e2'].quantile(0.50),
    'psf_e2_95': groups['psf_e2'].quantile(0.95),
    'band': groups['band'].first(),
})
visits_summary['psf_fwhm_95_05'] = np.sqrt(visits_summary['psf_fwhm_95']**2 - visits_summary['psf_fwhm_05']**2)
visits_summary['sys_50'] = np.sqrt(visits_summary['psf_fwhm_50']**2 + CAM_FWHM**2 - visits_summary['donut_blur_fwhm']**2)
visits_summary['psf_fwhm_model'] = np.sqrt(visits_summary['aos_fwhm']**2 + visits_summary['donut_blur_fwhm']**2)

In [ ]:
visits_summary['observation_reason']

In [ ]:
plt.figure()
plt.scatter(visits_summary['psf_fwhm_50'], visits_summary['psf_e_50'], s=2)
#plt.scatter(visits_summary['donut_blur_fwhm'], visits_summary['psf_e_50'], s=2)
#plt.scatter(visits_summary['psf_fwhm_95_05'], visits_summary['psf_e_50'], s=2)
plt.xlabel('Per-visit Median PSF FWHM (arcsec)')
plt.ylabel('Per-visit Mediain PSF e')

In [ ]:
visits_summary

In [ ]:
print(visits_summary['day_obs'].value_counts().sort_index().to_string())

In [ ]:
np.mean(visits_summary['day_obs'].value_counts()

In [ ]:
plt.figure()
plt.scatter(visits_summary['aos_fwhm'], visits_summary['psf_fwhm_95_05'], s=1)
#plt.xlim(0., 1.)
#plt.ylim(0., 1.)

In [ ]:
plt.figure()
plt.hist(visits_summary['psf_fwhm_50'], bins=41)

In [ ]:
selection = (visits_summary['band'] != "y") & (visits_summary['day_obs'] >= 20251026)
print(np.sum(selection))

plt.figure()
plt.hexbin(
    visits_summary["donut_blur_atm_fwhm"][selection], 
    visits_summary["psf_fwhm_50"][selection], 
    #visits_summary["psf_fwhm_zenith_500nm_50"][selection], 
    gridsize=40, mincnt=1, extent=(0.5, 2.0, 0.5, 2.0), cmap='viridis')
plt.colorbar(label='Density')
plt.plot(np.linspace(0.5, 2.0, 10), np.linspace(0.5, 2.0, 10), lw=2, ls='--', c='black')

x = np.linspace(0.5, 2.0, 100)
plt.plot(x, np.sqrt(x**2 + 0.4**2), c='red', lw=2, ls='--', label='0.4" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.5**2), c='orange', lw=2, ls='--', label='0.5" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.6**2), c='cyan', lw=2, ls='--', label='0.6" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.7**2), c='magenta', lw=2, ls='--', label='0.7" System Contribution')

plt.xlim(0.5, 1.8)
plt.ylim(0.5, 1.8)

#plt.xlabel("Donut Blur FWHM (arcsec)")
plt.xlabel("$\sqrt{\mathrm{DONUT\_BLUR\_FWHM}^2 - \mathrm{CAM\_FWHM}^2}$ (arcsec)")
#plt.ylabel("psf_fwhm_zenith_500nm")
plt.ylabel("PSF FWHM (arcsec)")
plt.title("Using Donut Blur as Proxy for Atmosphere Contribution")
plt.show()

## Zernikes

In [ ]:
import galsim

def getPsfGradPerZernike(
    diameter: float = 8.36,
    obscuration: float = 0.612,
    jmin: int = 4,
    jmax: int = 22,
) -> np.ndarray:
    """Get the gradient of the PSF FWHM with respect to each Zernike.

    This function takes no positional arguments. All parameters must be passed
    by name (see the list of parameters below).

    Parameters
    ----------
    diameter : float, optional
        The diameter of the telescope aperture, in meters.
        (the default, 8.36, corresponds to the LSST primary mirror)
    obscuration : float, optional
        Central obscuration of telescope aperture (i.e. R_outer / R_inner).
        (the default, 0.612, corresponds to the LSST primary mirror)
    jmin : int, optional
        The minimum Noll index, inclusive. Must be >= 0. (the default is 4)
    jmax : int, optional
        The max Zernike Noll index, inclusive. Must be >= jmin.
        (the default is 22.)

    Returns
    -------
    np.ndarray
        Gradient of the PSF FWHM with respect to the corresponding Zernike.
        Units are arcsec / micron.

    Raises
    ------
    ValueError
        If jmin is negative or jmax is less than jmin
    """
    # Check jmin and jmax
    if jmin < 0:
        raise ValueError("jmin cannot be negative.")
    if jmax < jmin:
        raise ValueError("jmax must be greater than jmin.")

    # Calculate the conversion factors
    conversion_factors = np.zeros(jmax + 1)
    for i in range(jmin, jmax + 1):
        # Set coefficients for this Noll index: coefs = [0, 0, ..., 1]
        # Note the first coefficient is Noll index 0, which does not exist and
        # is therefore always ignored by galsim
        coefs = [0] * i + [1]

        # Create the Zernike polynomial with these coefficients
        R_outer = diameter / 2
        R_inner = R_outer * obscuration
        Z = galsim.zernike.Zernike(coefs, R_outer=R_outer, R_inner=R_inner)

        # We can calculate the size of the PSF from the RMS of the gradient of
        # the wavefront. The gradient of the wavefront perturbs photon paths.
        # The RMS quantifies the size of the collective perturbation.
        # If we expand the wavefront gradient in another series of Zernike
        # polynomials, we can exploit the orthonormality of the Zernikes to
        # calculate the RMS from the Zernike coefficients.
        rms_tilt = np.sqrt(np.sum(Z.gradX.coef**2 + Z.gradY.coef**2) / 2)

        # Convert to arcsec per micron
        rms_tilt = np.rad2deg(rms_tilt * 1e-6) * 3600

        # Convert rms -> fwhm
        fwhm_tilt = 2 * np.sqrt(2 * np.log(2)) * rms_tilt

        # Save this conversion factor
        conversion_factors[i] = fwhm_tilt

    return conversion_factors[jmin:]


def convertZernikesToPsfWidth(
    zernikes: np.ndarray,
    diameter: float = 8.36,
    obscuration: float = 0.612,
    jmin: int = 4,
) -> np.ndarray:
    """Convert Zernike amplitudes to quadrature contribution to the PSF FWHM.

    Parameters
    ----------
    zernikes : np.ndarray
        Zernike amplitudes (in microns), starting with Noll index `jmin`.
        Either a 1D array of zernike amplitudes, or a 2D array, where each row
        corresponds to a different set of amplitudes.
    diameter : float
        The diameter of the telescope aperture, in meters.
        (the default, 8.36, corresponds to the LSST primary mirror)
    obscuration : float
        Central obscuration of telescope aperture (i.e. R_outer / R_inner).
        (the default, 0.612, corresponds to the LSST primary mirror)
    jmin : int
        The minimum Zernike Noll index, inclusive. Must be >= 0. The
        max Noll index is inferred from `jmin` and the length of `zernikes`.
        (the default is 4, which ignores piston, x & y offsets, and tilt.)

    Returns
    -------
    dFWHM: np.ndarray
        Quadrature contribution of each Zernike vector to the PSF FWHM
        (in arcseconds).

    Notes
    -----
    Converting Zernike amplitudes to their quadrature contributions to the PSF
    FWHM allows for easier physical interpretation of Zernike amplitudes and
    the performance of the AOS system.

    For example, image we have a true set of zernikes, [Z4, Z5, Z6], such that
    ConvertZernikesToPsfWidth([Z4, Z5, Z6]) = [0.1, -0.2, 0.3] arcsecs.
    These Zernike perturbations increase the PSF FWHM by
    sqrt[(0.1)^2 + (-0.2)^2 + (0.3)^2] ~ 0.37 arcsecs.

    If the AOS perfectly corrects for these perturbations, the PSF FWHM will
    not increase in size. However, imagine the AOS estimates zernikes, such
    that ConvertZernikesToPsfWidth([Z4, Z5, Z6]) = [0.1, -0.3, 0.4] arcsecs.
    These estimated Zernikes, do not exactly match the true Zernikes above.
    Therefore, the post-correction PSF will still be degraded with respect to
    the optimal PSF. In particular, the PSF FWHM will be increased by
    sqrt[(0.1 - 0.1)^2 + (-0.2 - (-0.3))^2 + (0.3 - 0.4)^2] ~ 0.14 arcsecs.

    This conversion depends on a linear approximation that begins to break down
    for RSS(dFWHM) > 0.20 arcsecs. Beyond this point, the approximation tends
    to overestimate the PSF degradation. In other words, if
    sqrt(sum( dFWHM^2 )) > 0.20 arcsec, it is likely that dFWHM is
    over-estimated. However, the point beyond which this breakdown begins
    (and whether the approximation over- or under-estimates dFWHM) can change,
    depending on which Zernikes have large amplitudes. In general, if you have
    large Zernike amplitudes, proceed with caution!
    Note that if the amplitudes Z_est and Z_true are large, this is okay, as
    long as |Z_est - Z_true| is small.

    For a notebook demonstrating where the approximation breaks down:
    https://gist.github.com/jfcrenshaw/24056516cfa3ce0237e39507674a43e1

    Raises
    ------
    ValueError
        If jmin is negative
    """
    # Check jmin
    if jmin < 0:
        raise ValueError("jmin cannot be negative.")

    # Calculate jmax from jmin and the length of the zernike array
    jmax = jmin + np.array(zernikes).shape[-1] - 1

    # Calculate the conversion factors for each zernike
    conversion_factors = getPsfGradPerZernike(
        jmin=jmin,
        jmax=jmax,
        diameter=diameter,
        obscuration=obscuration,
    )

    # Convert the Zernike amplitudes from microns to their quadrature
    # contribution to the PSF FWHM
    dFWHM = conversion_factors * zernikes

    return dFWHM

In [ ]:
def fetchZernikes(day_obs_min, day_obs_max):
    
    visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.z4,
    ccdvisit1_quicklook.z5,
    ccdvisit1_quicklook.z6,
    ccdvisit1_quicklook.z7,
    ccdvisit1_quicklook.z8,
    ccdvisit1_quicklook.z9,
    ccdvisit1_quicklook.z10,
    ccdvisit1_quicklook.z11,
    ccdvisit1_quicklook.z12,
    ccdvisit1_quicklook.z13,
    ccdvisit1_quicklook.z14,
    ccdvisit1_quicklook.z15,
    ccdvisit1_quicklook.z16,
    ccdvisit1_quicklook.z17,
    ccdvisit1_quicklook.z18,
    ccdvisit1_quicklook.z19,
    ccdvisit1_quicklook.z20,
    ccdvisit1_quicklook.z21,
    ccdvisit1_quicklook.z22,
    ccdvisit1_quicklook.z23,
    ccdvisit1_quicklook.z24,
    ccdvisit1_quicklook.z25,
    ccdvisit1_quicklook.z26,
    ccdvisit1_quicklook.z27,
    ccdvisit1_quicklook.z28,
    ccdvisit1.detector,
    visit1.visit_id
    FROM
    cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_{instrument}.ccdvisit1 AS ccdvisit1,
    cdb_{instrument}.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND ccdvisit1.detector IN (191, 195, 199, 200, 203) -- corner wavefront sensors
    -- AND ccdvisit1.detector IN (191, 192, 195, 196, 199, 200, 203, 204) -- corner wavefront sensors
    AND visit1.airmass > 0
    AND visit1.day_obs >= {day_obs_min} AND visit1.day_obs <= {day_obs_max}
    AND visit1.science_program in ('BLOCK-365', 'BLOCK-407', 'BLOCK-408', 'BLOCK-416', 'BLOCK-417', 'BLOCK-419');
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    return ccdvisits

In [ ]:
#day_obs_min = 20251026
#day_obs_max = 20260106
zernikes = fetchZernikes(day_obs_min, day_obs_max)

In [ ]:
np.unique(zernikes["detector"])

In [ ]:
zernike_columns = [f"z{i}" for i in range(4, 29)]
zernikes["zernikes"] = zernikes[zernike_columns].apply(
    lambda row: np.array(row.fillna(0.0).values, dtype=float), axis=1
)
zernikes["zernikes_fwhm"] = zernikes["zernikes"].apply(
    convertZernikesToPsfWidth
)

In [ ]:
zernikes_fwhm = np.vstack(zernikes["zernikes_fwhm"])
for ii, column in enumerate(zernike_columns):
    zernikes[f"{column}_fwhm"] = zernikes_fwhm[:,ii]

In [ ]:
zernikes["z4_fwhm"]

In [ ]:
zernikes.columns

In [ ]:
groups = zernikes.groupby('visit_id')
visits_summary_zernikes = pd.DataFrame({
    'z4_fwhm': groups['z4_fwhm'].mean(),
})

In [ ]:
visits_summary_zernikes

In [ ]:
#plt.figure()
#plt.hist(visits_summary_zernikes['z4_fwhm'], bins=np.linspace(-1., 1., 81))

In [ ]:
assert len(visits_summary_zernikes) == len(visits_summary)

visits_summary = pd.merge(visits_summary, visits_summary_zernikes, on="visit_id")

In [ ]:
visits_summary

## Correlation Analysis

In [ ]:
#plt.figure()
#plt.scatter(visits_summary["physical_rotator_angle"], visits_summary["psf_fwhm_95_05"], s=2, edgecolor='none')
#plt.scatter(visits_summary["physical_rotator_angle"], visits_summary["aos_fwhm"], s=2, edgecolor='none')
#plt.scatter(visits_summary["physical_rotator_angle"], visits_summary["sys_50"], s=2, edgecolor='none')
#plt.scatter(visits_summary["physical_rotator_angle"], visits_summary["psf_fwhm_50"], s=2, edgecolor='none')
#plt.hexbin(visits_summary["physical_rotator_angle"], visits_summary["psf_fwhm_95_05"], gridsize=20, mincnt=1,)
#plt.hexbin(visits_summary["physical_rotator_angle"], visits_summary["aos_fwhm"], gridsize=20, mincnt=1,)
#plt.hexbin(visits_summary["physical_rotator_angle"], visits_summary["sys_50"], gridsize=20, mincnt=1,)

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=True, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    hb = ax[ii].hexbin(visits_summary["physical_rotator_angle"], visits_summary[metric], gridsize=20, mincnt=1,)
    ax[ii].set_xlim(-90., 90.)
    ax[ii].set_ylim(0., 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel('Phys Rot Angle (deg)')
    ax[ii].set_ylabel('%s (arcsec)'%(metric))
    fig.colorbar(hb, ax=ax[ii], label='Counts')
    
#ax[0].legend()
#ax[-1].set_xlabel('%s (arsec)'%(metric))
#plt.suptitle('Physical Rotator Angle Changes')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.scatter(visits_summary["physical_rotator_angle"], visits_summary["altitude"], c=visits_summary["psf_fwhm_95_05"], vmin=0.3, vmax=0.6, s=5)
plt.colorbar()

In [ ]:
#plt.figure()
#plt.scatter(visits_summary["altitude"], visits_summary["psf_fwhm_95_05"], s=2, edgecolor='none')
#plt.scatter(visits_summary["altitude"], visits_summary["aos_fwhm"], s=2, edgecolor='none')
#plt.scatter(visits_summary["altitude"], visits_summary["sys_50"], s=2, edgecolor='none')
#plt.scatter(visits_summary["altitude"], visits_summary["psf_fwhm_50"], s=2, edgecolor='none')
#plt.hexbin(visits_summary["altitude"], visits_summary["psf_fwhm_95_05"], gridsize=20, mincnt=1,)
#plt.hexbin(visits_summary["altitude"], visits_summary["aos_fwhm"], gridsize=20, mincnt=1,)
#plt.hexbin(visits_summary["altitude"], visits_summary["sys_50"], gridsize=20, mincnt=1,)

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=True, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    hb = ax[ii].hexbin(visits_summary["altitude"], visits_summary[metric], gridsize=20, mincnt=1,)
    ax[ii].set_xlim(30., 90.)
    ax[ii].set_ylim(0., 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel('Altitude (deg)')
    ax[ii].set_ylabel('%s (arcsec)'%(metric))
    fig.colorbar(hb, ax=ax[ii], label='Counts')
    
#ax[0].legend()
#ax[-1].set_xlabel('%s (arsec)'%(metric))
#plt.suptitle('Physical Rotator Angle Changes')
plt.tight_layout()
plt.show()

In [ ]:
"""
selection = (visits_summary["altitude"] > 40.)

bins = np.linspace(0.2, 2, 21)

#metric = 'aos_fwhm'
metric = 'sys_50'
#metric = 'psf_fwhm_95_05'
#metric = 'psf_fwhm_50'

plt.figure()
plt.hist(visits_summary[metric][selection], bins=bins, density=True, histtype='step')
plt.hist(visits_summary[metric][~selection], bins=bins, density=True, histtype='step')
"""


threshold = 40
selection = (visits_summary["altitude"] < threshold)
bins = np.linspace(0.2, 2., 21)

#metric = 'aos_fwhm'
#metric = 'sys_50'
#metric = 'psf_fwhm_95_05'
#metric = 'psf_fwhm_50'

#plt.figure()
#plt.hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label='Filter Change (n = %i)'%(np.sum(selection)))
#plt.hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label='No Filter Change (n = %i)'%(np.sum(~selection)))
#plt.legend()

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=False, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    ax[ii].hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label='Altitude < %.f deg (n = %i)'%(threshold, np.sum(selection)))
    ax[ii].hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label='Altitude > %.f deg (n = %i)'%(threshold, np.sum(~selection)))
    ax[ii].set_xlim(0.2, 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel('%s (arsec)'%(metric))
    ax[ii].set_ylabel('PDF')
    
ax[0].legend()
plt.tight_layout()

In [ ]:
threshold = 0.
selection = (visits_summary["physical_rotator_angle"] < threshold)
bins = np.linspace(0.2, 2., 21)

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=False, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    ax[ii].hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label='Phys Rot Angle < %.f deg (n = %i)'%(threshold, np.sum(selection)))
    ax[ii].hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label='Phys Rot Angle > %.f deg (n = %i)'%(threshold, np.sum(~selection)))
    ax[ii].set_xlim(0.2, 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel('%s (arsec)'%(metric))
    ax[ii].set_ylabel('PDF')
    
ax[0].legend()
plt.tight_layout()
plt.show()

In [ ]:
selection = (visits_summary['day_obs'] < 20251126)

bins = np.linspace(20, 90, 21)

plt.figure()
plt.hist(visits_summary['altitude'][selection], bins=bins, density=True, histtype='step')
plt.hist(visits_summary['altitude'][~selection], bins=bins, density=True, histtype='step')

## DDF

In [ ]:
selection = visits_summary['target_name'].str.contains('ddf')

plt.figure()
for band in bands:
    band_selection = selection & (visits_summary["band"] == band)
    plt.scatter(
        visits_summary["exp_midpt_mjd"][selection & band_selection], 
        visits_summary["aos_fwhm"][selection & band_selection],
        c=colors[band], label=band,
        edgecolor='none', s=10
    )
plt.legend()

In [ ]:
selection = visits_summary['target_name'].str.contains('ddf')

plt.figure()
for band in bands:
    band_selection = selection & (visits_summary["band"] == band)
    plt.scatter(visits_summary['altitude'][selection & band_selection], visits_summary[''][selection & band_selection], c=colors[band], label=band,)
plt.legend()

In [ ]:
visits_summary[selection]['day_obs'].value_counts().sort_index()

## Differences

In [ ]:
a = [1, 10, 100, 1000]
np.diff(a)

In [ ]:
selection_filter_change = (visits_summary["band"][1:].values != visits_summary["band"][0:-1].values)
selection_consecutive = np.diff(visits_summary['seq_num']) == 1
selection = (selection_consecutive & selection_filter_change)

bins = np.linspace(0.2, 2.0, 41)

#metric = 'aos_fwhm'
#metric = 'sys_50'
#metric = 'psf_fwhm_95_05'
#metric = 'psf_fwhm_50'

#plt.figure()
#plt.hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label='Filter Change (n = %i)'%(np.sum(selection)))
#plt.hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label='No Filter Change (n = %i)'%(np.sum(~selection)))
#plt.legend()

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=False, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    ax[ii].hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label='Filter Change (n = %i)'%(np.sum(selection)))
    ax[ii].hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label='No Filter Change (n = %i)'%(np.sum(~selection)))
    ax[ii].set_xlim(0.2, 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel('%s (arsec)'%(metric))
    ax[ii].set_ylabel('PDF')
    
ax[0].legend()
#ax[-1].set_xlabel('%s (arsec)'%(metric))
#plt.suptitle('Physical Rotator Angle Changes')
plt.tight_layout()

In [ ]:
"""
selection_consecutive = np.diff(visits_summary['seq_num']) == 1

#metric = 'aos_fwhm'
#metric = 'sys_50'
metric = 'psf_fwhm_95_05'
#metric = 'psf_fwhm_50'

plt.figure()
plt.scatter(np.diff(visits_summary["altitude"])[selection_consecutive], visits_summary[metric][1:][selection_consecutive], s=1)
#plt.scatter(np.diff(visits_summary["physical_rotator_angle"])[selection_consecutive], visits_summary[metric][1:][selection_consecutive], s=1)
"""

threshold = 10.

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=False, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    ax[ii].axvspan(-1. * threshold, threshold, color='0.9')
    ax[ii].scatter(np.diff(visits_summary["altitude"])[selection_consecutive], visits_summary[metric][1:][selection_consecutive], s=2, edgecolor='none')
    ax[ii].set_xlim(-45, 45.)
    ax[ii].set_ylim(0.2, 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel(r'$\Delta$ altitude (deg)')
    ax[ii].set_ylabel('%s (arsec)'%(metric))

plt.tight_layout()

In [ ]:
threshold = 10. 

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=False, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    ax[ii].axvspan(-1. * threshold, threshold, color='0.9')
    ax[ii].scatter(np.diff(visits_summary["physical_rotator_angle"])[selection_consecutive], visits_summary[metric][1:][selection_consecutive], s=2, edgecolor='none')
    ax[ii].set_xlim(-100, 100.)
    ax[ii].set_ylim(0.2, 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel(r'$\Delta$ phys rot angle (deg)')
    ax[ii].set_ylabel('%s (arsec)'%(metric))

plt.tight_layout()
plt.show()

In [ ]:
selection_consecutive = np.diff(visits_summary['seq_num']) == 1
#selection_slew = (np.fabs(np.diff(visits_summary["altitude"])) > 10)
selection_slew = (np.fabs(np.diff(visits_summary["physical_rotator_angle"])) > 20.)

selection = (selection_slew & selection_consecutive)

bins = np.linspace(0.2, 2.0, 41)

metric = 'aos_fwhm'
#metric = 'sys_50'
#metric = 'psf_fwhm_95_05'
#metric = 'psf_fwhm_50'

plt.figure()
plt.hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label='Large Slew (n = %i)'%(np.sum(selection)))
plt.hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label='No large Slew (n = %i)'%(np.sum(~selection)))
plt.legend()

In [ ]:
selection_consecutive = np.diff(visits_summary['seq_num']) == 1
#selection_slew = (np.fabs(np.diff(visits_summary["altitude"])) > 10)
threshold = 10.
selection_slew = (np.fabs(np.diff(visits_summary["physical_rotator_angle"])) > threshold)

selection = (selection_slew & selection_consecutive)

bins = np.linspace(0.2, 2.0, 41)

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=False, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    ax[ii].hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label=r'$|\Delta~\mathrm{phys~rot~angle}| > %.f$ deg (n = %i)'%(threshold , np.sum(selection)))
    ax[ii].hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label=r'$|\Delta~\mathrm{phys~rot~angle}| < %.f$ deg (n = %i)'%(threshold, np.sum(~selection)))
    ax[ii].set_xlim(0.2, 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel('%s (arsec)'%(metric))
    ax[ii].set_ylabel('PDF')
    
ax[0].legend()
#ax[-1].set_xlabel('%s (arsec)'%(metric))
#plt.suptitle('Physical Rotator Angle Changes')
plt.tight_layout()

In [ ]:
selection_consecutive = np.diff(visits_summary['seq_num']) == 1
threshold = 10
selection_slew = (np.fabs(np.diff(visits_summary["altitude"])) > threshold)
#threshold = 20.
#selection_slew = (np.fabs(np.diff(visits_summary["physical_rotator_angle"])) > threshold)

selection = (selection_slew & selection_consecutive)

bins = np.linspace(0.2, 2.0, 41)

metrics = ('aos_cam_fwhm', 'sys_50', 'psf_fwhm_95_05')
fig, ax = plt.subplots(len(metrics), sharex=False, figsize=(6, 8))

for ii, metric in enumerate(metrics):
    print(ii, metric)
    ax[ii].hist(visits_summary[metric][1:][selection], bins=bins, histtype='step', density=True, label=r'$|\Delta~\mathrm{altitude}| > %.f$ deg (n = %i)'%(threshold , np.sum(selection)))
    ax[ii].hist(visits_summary[metric][1:][~selection], bins=bins, histtype='step', density=True, label=r'$|\Delta~\mathrm{altitude}| < %.f$ deg (n = %i)'%(threshold, np.sum(~selection)))
    ax[ii].set_xlim(0.2, 2.)
    #ax[ii].set_title(metric)
    ax[ii].set_xlabel('%s (arsec)'%(metric))

ax[0].legend()
#ax[-1].set_xlabel('%s (arsec)'%(metric))
#plt.suptitle('Physical Rotator Angle Changes')
plt.tight_layout()
plt.show()

## Inserted Visits

In [ ]:
np.sum(visits_summary['observation_reason'].str.contains('filter_change_close_loop'))

In [ ]:
import scipy

In [ ]:
# Identify the first visit of filter_change_close_loop sequence
condition = visits_summary['observation_reason'].str.contains('filter_change_close_loop') #& visits_summary['science_program'] == 'BLOCK-419'

label, num_features = scipy.ndimage.label(condition)
index_filter_change_close_loop = np.array([label.tolist().index(_) for _ in range(1, num_features)])

In [ ]:
index_filter_change_close_loop

In [ ]:
visits_summary['observation_reason'].iloc[index_filter_change_close_loop]

In [ ]:
visits_summary['science_program'].iloc[index_filter_change_close_loop]

In [ ]:
print(len(index_filter_change_close_loop))

In [ ]:
plt.figure()

nvisits = 16

#metric = 'aos_fwhm'
metric = 'z4_fwhm'
#metric = 'psf_fwhm_50'

markers = ['o', 'v', '^', 's', 'p', 'h', 'H', '+', 'x', 'd']

for ii in range(0, len(index_filter_change_close_loop)):
    #print(ii)
    #print(visits_summary['target_name'].iloc[index_filter_change_close_loop[ii]])
    color = 'red' if 'ddf' in visits_summary['target_name'].iloc[index_filter_change_close_loop[ii]] else 'black'
    
    #print(visits_summary['target_name'].iloc[index_filter_change_close_loop[ii] + np.arange(nvisits)] == visits_summary['target_name'].iloc[index_filter_change_close_loop[ii]])
    delta_altitude = np.fabs(
        visits_summary['altitude'].iloc[index_filter_change_close_loop[ii] + np.arange(nvisits)] - visits_summary['altitude'].iloc[index_filter_change_close_loop[ii]]
    )
    delta_physical_rotator_angle = np.fabs(
        visits_summary['physical_rotator_angle'].iloc[index_filter_change_close_loop[ii] + np.arange(nvisits)] - visits_summary['physical_rotator_angle'].iloc[index_filter_change_close_loop[ii]]
    )
    same_filter = (visits_summary['band'].iloc[index_filter_change_close_loop[ii] + np.arange(nvisits)] == visits_summary['band'].iloc[index_filter_change_close_loop[ii]])
    selection = (delta_altitude < 10.) & (delta_physical_rotator_angle < 10.) & (same_filter)

    #print(same_filter[selection])
    #print(delta_altitude[selection])
    #print(delta_physical_rotator_angle[selection])
    #print(visits_summary['band'].iloc[index_filter_change_close_loop[ii] + np.arange(nvisits)])
    #print(same_filter)
    #print(delta_altitude < 10.)
    #print(delta_physical_rotator_angle < 10.)
    
    x = np.arange(nvisits) + np.random.uniform(-0.1, 0.1)
    y = visits_summary[metric].iloc[index_filter_change_close_loop[ii] + np.arange(nvisits)]
    #y = visits_summary[metric].iloc[index_filter_change_close_loop[ii] + np.arange(nvisits)] - visits_summary[metric].iloc[index_filter_change_close_loop[ii] - 1]
    #plt.scatter(x[selection], y[selection], marker=markers[ii % len(markers)])
    plt.plot(x[selection], y[selection], lw=1)
plt.xlabel('Visits after filter change (zero indexed)')
plt.ylabel(metric)
#plt.ylabel('Change in aos_fwhm relative to visit before filter change')
#plt.ylim(-1.2, 1.2)

In [ ]:
np.arange(-10, nvisits)

In [ ]:
np.arange(6)

## 

## Individual Day Summary

In [ ]:
day_obs = 20260106
selection = (visits_summary['day_obs'] == day_obs)

print(np.sum(selection))

fig, ax = plt.subplots(3, sharex=True, figsize=(16, 10))

ax[0].errorbar(visits_summary['seq_num'][selection], 
               visits_summary['psf_fwhm_50'][selection],
               yerr=np.array([
                   visits_summary['psf_fwhm_50'][selection] - visits_summary['psf_fwhm_05'][selection], 
                   visits_summary['psf_fwhm_95'][selection] - visits_summary['psf_fwhm_50'][selection],
               ]), 
               fmt='_', label='PSF FWHM: 5, 50, 95 percentile')
ax[0].errorbar(visits_summary['seq_num'][selection], 
               visits_summary['psf_e_50'][selection],
               yerr=np.array([
                   visits_summary['psf_e_50'][selection] - visits_summary['psf_e_05'][selection], 
                   visits_summary['psf_e_95'][selection] - visits_summary['psf_e_50'][selection],
               ]), 
               fmt='_', label='PSF e: 5, 50, 95 percentile')
#ax[0].scatter(visits_summary['seq_num'], visits_summary['psf_fwhm_95'])
#ax[0].scatter(visits_summary['seq_num'], visits_summary['psf_fwhm_50'])
#ax[0].scatter(visits_summary['seq_num'], visits_summary['psf_fwhm_05'])
ax[0].scatter(visits_summary['seq_num'][selection], visits_summary['psf_fwhm_95_05'][selection], c='red', marker='x', label='sqrt(fwhm_95^2 - fwhm_5^2)')
ax[0].scatter(visits_summary['seq_num'][selection], visits_summary['donut_blur_fwhm'][selection], c='black', marker='o', label='Donut Blur')
ax[0].scatter(visits_summary['seq_num'][selection], visits_summary['psf_fwhm_model'][selection], c='0.8', marker='o', label='Model')
ax[0].scatter(visits_summary['seq_num'][selection], visits_summary['aos_fwhm'][selection], c='green', marker='+', label='AOS Residual')
ax[0].set_ylabel('Delivered Image Quality Metric\n(arcsec)')
#ax[0].scatter(visit_seq_num, np.sqrt(visit_donut_blur_fwhm**2 + visit_psf_fwhm_95_05**2), c='green', marker='+', label='AOS Residual')
ax[0].legend(loc='upper right')
ax[0].set_ylim(0., 2.0)
ax[0].grid(True)

for band in bands:
    band_selection = selection & (visits_summary["band"] == band)
    ax[1].scatter(visits_summary['seq_num'][band_selection], visits_summary['altitude'][band_selection], marker='.', s=50, c=colors[band], label=band)
#ax[1].scatter(visits_summary['seq_num'], visits_summary['altitude'], marker='.', s=10, c='black')
ax[1].set_ylabel('Elevation\n(deg)')
ax[1].grid(True)
ax[1].legend()

for band in bands:
    band_selection = selection & (visits_summary["band"] == band)
    ax[2].scatter(visits_summary['seq_num'][band_selection], visits_summary['physical_rotator_angle'][band_selection], marker='.', s=50, c=colors[band], label=band)
#ax[2].scatter(visits_summary['seq_num'], visits_summary['physical_rotator_angle'], marker='.', s=10, c='black')
ax[2].set_ylabel('Phys Rot Angle\n(deg)')
ax[2].grid(True)

ax[-1].set_xlabel('seq_num')

plt.suptitle('day_obs = %s'%(day_obs))

#plt.suptitle('day_obs = %s'%(day_obs_min))
plt.subplots_adjust(hspace=0)
plt.tight_layout()
plt.show()

In [ ]:
selection = np.isin(visits_summary['day_obs'], [20251026])

bins = np.linspace(0., 1.5, 40)

plt.figure()
#plt.hist(visits_summary['aos_fwhm'], bins=bins, density=True, histtype='step')
#plt.hist(visits_summary['aos_fwhm'][selection], bins=bins, density=True, histtype='step')
plt.hist(visits_summary['psf_fwhm_95_05'], bins=bins, density=True, histtype='step', lw=2, label='All (nvisits = %i)'%(len(selection)))
plt.hist(visits_summary['psf_fwhm_95_05'][selection], bins=bins, density=True, histtype='step', lw=2, label='20251026 (nvisits = %i)'%(np.sum(selection)))
plt.xlabel('sqrt(fwhm_95^2 - fwhm_5^2) (arcsec)')
plt.xlim(0., 1.5)
plt.legend()

In [ ]:
plt.figure()
plt.scatter(visits_summary['aos_fwhm'], visits_summary['psf_fwhm_95_05'], s=1)
plt.xlim(0.2, 1.2)
plt.ylim(0.2, 1.2)

## Day Obs Summary

In [ ]:
#for key in visits_summary.keys():
#    visits_summary[key] = pd.to_numeric(visits_summary[key], errors="coerce")

# Exclude y due to issues with estimating donut blur
selection = (visits_summary['band'] != "y")

groups = visits_summary[selection].groupby('day_obs')
day_obs_summary = pd.DataFrame({
    'day_obs': groups['day_obs'].first(),
    'n_visits': groups['day_obs'].count(),
    'psf_fwhm_95_05_low': groups['psf_fwhm_95_05'].quantile(0.10),
    'psf_fwhm_95_05_50': groups['psf_fwhm_95_05'].quantile(0.50),
    'psf_fwhm_95_05_high': groups['psf_fwhm_95_05'].quantile(0.90),
    'aos_fwhm_low': groups['aos_fwhm'].quantile(0.10),
    'aos_fwhm_50': groups['aos_fwhm'].quantile(0.50),
    'aos_fwhm_high': groups['aos_fwhm'].quantile(0.90),
    'aos_cam_fwhm_low': groups['aos_cam_fwhm'].quantile(0.10),
    'aos_cam_fwhm_50': groups['aos_cam_fwhm'].quantile(0.50),
    'aos_cam_fwhm_high': groups['aos_cam_fwhm'].quantile(0.90),
    'sys_50_low': groups['sys_50'].quantile(0.10),
    'sys_50_50': groups['sys_50'].quantile(0.50),
    'sys_50_high': groups['sys_50'].quantile(0.90),
    'psf_e_50_low': groups['psf_e_50'].quantile(0.10),
    'psf_e_50_50': groups['psf_e_50'].quantile(0.50),
    'psf_e_50_high': groups['psf_e_50'].quantile(0.90),
    'psf_e1_50_low': groups['psf_e1_50'].quantile(0.10),
    'psf_e1_50_50': groups['psf_e1_50'].quantile(0.50),
    'psf_e1_50_high': groups['psf_e1_50'].quantile(0.90),
    'psf_e2_50_low': groups['psf_e2_50'].quantile(0.10),
    'psf_e2_50_50': groups['psf_e2_50'].quantile(0.50),
    'psf_e2_50_high': groups['psf_e2_50'].quantile(0.90),
    'psf_fwhm_50_low': groups['psf_fwhm_50'].quantile(0.10),
    'psf_fwhm_50_50': groups['psf_fwhm_50'].quantile(0.50),
    'psf_fwhm_50_high': groups['psf_fwhm_50'].quantile(0.90),
    'donut_blur_atm_fwhm_low': groups['donut_blur_atm_fwhm'].quantile(0.10),
    'donut_blur_atm_fwhm_50': groups['donut_blur_atm_fwhm'].quantile(0.50),
    'donut_blur_atm_fwhm_high': groups['donut_blur_atm_fwhm'].quantile(0.90),
})

In [ ]:
selection = np.tile(True, len(day_obs_summary))

xticks = np.arange(len(day_obs_summary[selection]))

plt.figure(figsize=(10, 6))
plt.errorbar(xticks - 0.15, day_obs_summary['psf_fwhm_95_05_50'][selection],
             yerr=np.array([day_obs_summary['psf_fwhm_95_05_50'][selection] - day_obs_summary['psf_fwhm_95_05_low'][selection], 
                            day_obs_summary['psf_fwhm_95_05_high'][selection] - day_obs_summary['psf_fwhm_95_05_50'][selection]]), 
             fmt='_', c='tab:blue', label='sqrt(fwhm_95^2 - fwhm_5^2): 10th, 50th, 90th Percentile')
#plt.errorbar(xticks + 0.2, day_obs_summary['aos_fwhm_50'][selection],
#             yerr=np.array([day_obs_summary['aos_fwhm_50'][selection] - day_obs_summary['aos_fwhm_low'][selection], 
#                            day_obs_summary['aos_fwhm_high'][selection] - day_obs_summary['aos_fwhm_50'][selection]]), 
#             fmt='_', c='magenta', label='aos_fwhm: 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0., day_obs_summary['aos_cam_fwhm_50'][selection],
             yerr=np.array([day_obs_summary['aos_cam_fwhm_50'][selection] - day_obs_summary['aos_cam_fwhm_low'][selection], 
                            day_obs_summary['aos_cam_fwhm_high'][selection] - day_obs_summary['aos_cam_fwhm_50'][selection]]), 
             fmt='_', c='tab:purple', label='sqrt(aos_fwhm^2 + cam_fwhm^2): 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0.15, day_obs_summary['sys_50_50'][selection],
             yerr=np.array([day_obs_summary['sys_50_50'][selection] - day_obs_summary['sys_50_low'][selection], 
                            day_obs_summary['sys_50_high'][selection] - day_obs_summary['sys_50_50'][selection]]), 
             fmt='_', c='tab:red', label='sqrt(fwhm_50^2 - (donut_blur^2 - cam_fwhm^2)): 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0., day_obs_summary['psf_e_50_50'][selection],
             yerr=np.array([day_obs_summary['psf_e_50_50'][selection] - day_obs_summary['psf_e_50_low'][selection], 
                            day_obs_summary['psf_e_50_high'][selection] - day_obs_summary['psf_e_50_50'][selection]]), 
             fmt='_', c='tab:orange', label='psf_e: 10th, 50th, 90th Percentile')


plt.errorbar(xticks - 0.075, day_obs_summary['psf_fwhm_50_50'][selection],
             yerr=np.array([day_obs_summary['psf_fwhm_50_50'][selection] - day_obs_summary['psf_fwhm_50_low'][selection], 
                            day_obs_summary['psf_fwhm_50_high'][selection] - day_obs_summary['psf_fwhm_50_50'][selection]]), 
             fmt='_', c='black', label='fwhm_50: 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0.075, day_obs_summary['donut_blur_atm_fwhm_50'][selection],
             yerr=np.array([day_obs_summary['donut_blur_atm_fwhm_50'][selection] - day_obs_summary['donut_blur_atm_fwhm_low'][selection], 
                            day_obs_summary['donut_blur_atm_fwhm_high'][selection] - day_obs_summary['donut_blur_atm_fwhm_50'][selection]]), 
             fmt='_', c='0.5', label='sqrt(donut_blur^2 - cam_fwhm^2): 10th, 50th, 90th Percentile')

plt.legend()
plt.xticks(xticks, day_obs_summary['day_obs'][selection], rotation=90)
plt.ylim(0, plt.ylim()[-1])
#for y in [0.04, 0.2, 0.4, 0.6]:
#    plt.axhline(y, c='0.75', ls='--', zorder=0)

plt.axhline(1., c='0.25', ls=':', zorder=0)
plt.axhline(0.45, c='red', ls=':', zorder=0)
plt.axhline(0.04, c='orange', ls=':', zorder=0)

plt.ylabel('Image Quality Contribution (arcsec)')
plt.tight_layout()
#plt.savefig('system_diq_timeline.pdf')

In [ ]:
from astropy.time import Time

def dayObsToTime(day_obs):
    year = str(day_obs)[0:4]
    month = str(day_obs)[4:6]
    day = str(day_obs)[6:8]
    time = Time('%s-%s-%s 12:00:00'%(year, month, day), format='iso')
    return time

selection = np.tile(True, len(day_obs_summary))

#xticks = np.arange(len(day_obs_summary[selection]))
xticks = np.array([dayObsToTime(_).mjd for _ in day_obs_summary['day_obs'][selection]])

print(xticks)

fig, ax = plt.subplots(figsize=(10, 6), dpi=200)
a = np.linspace(0., 1., 10)

variation = ax.errorbar(
    xticks - 0.15, day_obs_summary['psf_fwhm_95_05_50'][selection],
    yerr=np.array([
        day_obs_summary['psf_fwhm_95_05_50'][selection] - day_obs_summary['psf_fwhm_95_05_low'][selection], 
        day_obs_summary['psf_fwhm_95_05_high'][selection] - day_obs_summary['psf_fwhm_95_05_50'][selection]
    ]), 
    fmt='_', c='tab:blue', label='Variation Across FoV'
    #label='sqrt(fwhm_95^2 - fwhm_5^2)'
)
aos = ax.errorbar(
    xticks + 0., day_obs_summary['aos_cam_fwhm_50'][selection],
    yerr=np.array([day_obs_summary['aos_cam_fwhm_50'][selection] - day_obs_summary['aos_cam_fwhm_low'][selection], 
                   day_obs_summary['aos_cam_fwhm_high'][selection] - day_obs_summary['aos_cam_fwhm_50'][selection]]), 
    fmt='_', c='tab:purple', label='Optics + Camera Diffusion'
    #label='sqrt(aos_fwhm^2 + cam_fwhm^2)'
)
sys = ax.errorbar(
    xticks + 0.15, day_obs_summary['sys_50_50'][selection],
    yerr=np.array([day_obs_summary['sys_50_50'][selection] - day_obs_summary['sys_50_low'][selection], 
                   day_obs_summary['sys_50_high'][selection] - day_obs_summary['sys_50_50'][selection]]), 
    fmt='_', c='tab:red', label='Delivered - Estimated Atmosphere'
    #label='sqrt(fwhm_50^2 - (donut_blur^2 - cam_fwhm^2))'
)

ellipticity = ax.errorbar(
    xticks + 0., day_obs_summary['psf_e_50_50'][selection],
    yerr=np.array([day_obs_summary['psf_e_50_50'][selection] - day_obs_summary['psf_e_50_low'][selection], 
                   day_obs_summary['psf_e_50_high'][selection] - day_obs_summary['psf_e_50_50'][selection]]), 
    fmt='_', c='tab:orange', label='PSF Ellipticity'
    #label='psf_e'
)
"""
ellipticity_1 = ax.errorbar(
    xticks - 0.15, day_obs_summary['psf_e1_50_50'][selection],
    yerr=np.array([day_obs_summary['psf_e1_50_50'][selection] - day_obs_summary['psf_e1_50_low'][selection], 
                   day_obs_summary['psf_e1_50_high'][selection] - day_obs_summary['psf_e1_50_50'][selection]]), 
    fmt='_', c='tab:orange',
    #label='psf_e'
)
ellipticity_2 = ax.errorbar(
    xticks + 0.15, day_obs_summary['psf_e2_50_50'][selection],
    yerr=np.array([day_obs_summary['psf_e2_50_50'][selection] - day_obs_summary['psf_e2_50_low'][selection], 
                   day_obs_summary['psf_e2_50_high'][selection] - day_obs_summary['psf_e2_50_50'][selection]]), 
    fmt='_', c='tab:orange',
    #label='psf_e'
)
"""

psf = ax.errorbar(
    xticks - 0.075, day_obs_summary['psf_fwhm_50_50'][selection],
    yerr=np.array([day_obs_summary['psf_fwhm_50_50'][selection] - day_obs_summary['psf_fwhm_50_low'][selection], 
                   day_obs_summary['psf_fwhm_50_high'][selection] - day_obs_summary['psf_fwhm_50_50'][selection]]), 
    fmt='_', c='black', label='Delivered Median PSF FWHM' 
    #label='fwhm_50'
)
atm = ax.errorbar(
    xticks + 0.075, day_obs_summary['donut_blur_atm_fwhm_50'][selection],
    yerr=np.array([day_obs_summary['donut_blur_atm_fwhm_50'][selection] - day_obs_summary['donut_blur_atm_fwhm_low'][selection], 
                   day_obs_summary['donut_blur_atm_fwhm_high'][selection] - day_obs_summary['donut_blur_atm_fwhm_50'][selection]]), 
    fmt='_', c='0.5', label='Estimated Atmosphere'
    #label='sqrt(donut_blur^2 - cam_fwhm^2)'
)


first_legend = ax.legend(handles=[variation, aos, sys], loc='upper left', title='Estimated System Contribution')
#first_legend = ax.legend(handles=[variation, aos], loc='upper left', title='Estimated System Contribution')
second_legend = ax.legend(handles=[psf, atm, ellipticity], loc='upper right', title='Measured Quantities')
ax.add_artist(first_legend)

ax.set_xticks(xticks, day_obs_summary['day_obs'][selection], rotation=90)
#ax.set_ylim(0, plt.ylim()[-1])
#ax.set_ylim(0, 2.5)
ax.set_ylim(0., 2.0)
#for y in [0.04, 0.2, 0.4, 0.6]:
#    plt.axhline(y, c='0.75', ls='--', zorder=0)

#ax.axhline(1., c='0.25', ls=':', zorder=0)
ax.axhline(0.45, c='red', ls=':', zorder=0)
ax.axhline(0.04, c='orange', ls=':', zorder=0)

ax.set_ylabel('Image Quality Metric (arcsec)')
plt.title('10th, 50th, 90th Percentiles of Per-visit Quantities for Each Night')
plt.grid()
fig.tight_layout()
#plt.savefig('system_diq_timeline.pdf')

In [ ]:
xticks = np.arange(len(day_obs_summary))

plt.figure(figsize=(10, 6))
plt.errorbar(xticks, day_obs_summary['aos_fwhm_50'],
             yerr=np.array([day_obs_summary['aos_fwhm_50'] - day_obs_summary['aos_fwhm_05'], 
                            day_obs_summary['aos_fwhm_95'] - day_obs_summary['aos_fwhm_50']]), 
             fmt='_')
plt.xticks(xticks, day_obs_summary['day_obs'], rotation=90)
plt.ylabel('aos_fwhm')
plt.tight_layout()

In [ ]:
plt.figure()
for band in bands:
    selection = (visits_summary["band"] == band)
    plt.scatter(visits_summary['donut_blur_fwhm'][selection], visits_summary['psf_fwhm_50'][selection], s=2, c=colors[band], label=band)
plt.xlim(0.5, 2.)
plt.ylim(0.5, 2.)
plt.legend(markerscale=3)
plt.xlabel('donut_blur_fwhm (arcsec)')
plt.ylabel('median_psf_fwhm (arcsec)')

In [ ]:
bins=np.linspace(-0.5, 0.5, 50)

plt.figure()
#for band in bands:
for band in ['g', 'r', 'i', 'z']:
    selection = (visits_summary["band"] == band) & (visits_summary['psf_fwhm_50'] < 2.0)
    plt.hist(visits_summary['donut_blur_fwhm'][selection] - visits_summary['psf_fwhm_50'][selection], bins=bins, density=True, histtype='step', color=colors[band], label=band, lw=2)
plt.legend()
plt.xlabel('median_psf_fwhm - donut_blur_fwhm (arcsec)')
plt.ylabel('PDF')